## Wide Residual Net

 - Training Dataset:  RandAugment, N=2, M=5
   Validation with Unaugmented Data
 - Sagemaker Notebook must be of type, conda_pytorch_p36
 
#### Install Requirements

In [8]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

In [9]:
!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


#### Train the Model

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    train.batch_size 128 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_RA_2_5 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00 \
    scheduler.epochs 400

[2020-07-16 21:55:09] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_2_5
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
    a

#### Refine the Model

In [ ]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR

[2020-07-18 22:09:44] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: 

#### Evaluate Test Datasets

In [10]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00/test_results_0400_cifar10

[2020-07-19 10:55:43] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:25<00:00,  3.04it/s]
[2020-07-19 10:56:10] __main__ INFO: Elapsed 25.96
[2020-07-19 10:56:10] __main__ INFO: Loss 0.3968 Accuracy 0.9109


In [11]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - without training on unaugmented data
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00/test_results_0400_cifar101

[2020-07-19 10:56:23] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:05<00:00,  2.86it/s]
[2020-07-19 10:56:29] __main__ INFO: Elapsed 5.60
[2020-07-19 10:56:29] __main__ INFO: Loss 0.8234 Accuracy 0.8180


In [12]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00_resume400_50/test_results_0050_cifar10

[2020-07-19 10:56:43] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:26<00:00,  3.03it/s]
[2020-07-19 10:57:11] __main__ INFO: Elapsed 26.11
[2020-07-19 10:57:11] __main__ INFO: Loss 0.2752 Accuracy 0.9376


In [13]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00_resume400_50/test_results_0050_cifar101

[2020-07-19 10:57:27] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:05<00:00,  2.83it/s]
[2020-07-19 10:57:34] __main__ INFO: Elapsed 5.65
[2020-07-19 10:57:34] __main__ INFO: Loss 0.6268 Accuracy 0.8600


#### Record the Results

In [14]:
import pandas as pd
model = 'wrn_28_10_ra_2_5_c10val'
model_refined = model + '_refined400'

a = pd.Series([model, 400, 'cifar10', 0.3968, 0.9109])
c = pd.Series([model, 400, 'cifar10.1', 0.8234, 0.8180])

e = pd.Series([model_refined, 50, 'cifar10.1', 0.6268, 0.8600])
f = pd.Series([model_refined, 50, 'cifar10', 0.2752, 0.9376])
               
df_results = pd.concat([a,c,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 92.5 if row[2] == 'cifar10' else 84.9), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (92.0, 93.0) if row[2] == 'cifar10' else (83.2, 86.4)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/' + model + '/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,wrn_28_10_ra_2_5_c10val,400,cifar10,0.3968,0.9109,92.5,"(92.0, 93.0)"
1,wrn_28_10_ra_2_5_c10val,400,cifar10.1,0.8234,0.818,84.9,"(83.2, 86.4)"
2,wrn_28_10_ra_2_5_c10val_refined400,50,cifar10.1,0.6268,0.86,84.9,"(83.2, 86.4)"
3,wrn_28_10_ra_2_5_c10val_refined400,50,cifar10,0.2752,0.9376,92.5,"(92.0, 93.0)"


In [15]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/wrn_28_10_ra_2_5_c10val'
path = '/home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5_c10val'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)